# Data Preparation and EDA for IoT23 dataset

In [2]:
import os 
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import csv
from numpy import array
from numpy import argmax
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
data= pd.read_csv('iot23_combined_1M.csv')

C:\Users\DELL\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
data.head()  

,Unnamed: 0,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,0,1.536227e+09,CeqqKl3hyLQmO8LK98,192.168.100.111,17576.0,78.1.220.212,8081.0,tcp,-,3e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
1,1,1.536227e+09,C2oHQWo1EFGH8D9x7,192.168.100.111,17576.0,152.84.7.111,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
2,2,1.536227e+09,CJLVjs4BByG04mczXc,192.168.100.111,17576.0,173.36.41.67,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
3,3,1.536227e+09,C0z4uS9AWHDH2s4S7,192.168.100.111,17576.0,87.13.21.104,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS
4,4,1.536227e+09,CxbNVk3liFNUIlqSPi,192.168.100.111,17576.0,99.110.163.140,8081.0,tcp,-,2e-06,...,S0,-,-,0.0,S,2.0,80.0,0.0,0.0,POHS


In [5]:
del data['Unnamed: 0']

# Cleaning Process start! 

# dropping unncessary attributes ! 

In [6]:
df= data.drop(columns=['uid','id.orig_h','id.resp_h','local_orig','local_resp'])

In [7]:
df.head()

,ts,id.orig_p,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1.536227e+09,17576.0,8081.0,tcp,-,3e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHS
1,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHS
2,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHS
3,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHS
4,1.536227e+09,17576.0,8081.0,tcp,-,2e-06,0,0,S0,0.0,S,2.0,80.0,0.0,0.0,POHS


Let us separate the target and independent variables 

okkkk... so there are 7 target variables with very few samples.. we would like to remove all those 

In [230]:
df.drop(df[df['label'] == 'C&C-HeartBeat' ].index, inplace = True)

In [231]:
df.drop(df[df['label'] == 'Attack' ].index, inplace = True)

In [232]:
df.drop(df[df['label'] == 'C&C-FileDownload' ].index, inplace = True)

In [233]:
df.drop(df[df['label'] == 'C&C-Mirai' ].index, inplace = True)

In [234]:
df.drop(df[df['label'] == 'C&C-HeartBeat-FileDownload' ].index, inplace = True)

In [235]:
df.drop(df[df['label'] == 'FileDownload' ].index, inplace = True)

In [236]:
df.drop(df[df['label'] == 'C&C-Torii' ].index, inplace = True)

In [8]:
df['orig_bytes'] = df['orig_bytes'].str.replace('-','0')
df['resp_bytes'] = df['resp_bytes'].str.replace('-','0')
df['duration'] = df['duration'].str.replace('-','0')

In [9]:
df.isnull().sum()

ts                    0
id.orig_p             0
id.resp_p             0
proto                 0
service               0
duration         360448
orig_bytes       360448
resp_bytes       360448
conn_state            0
missed_bytes          0
history               0
orig_pkts             0
orig_ip_bytes         0
resp_pkts             0
resp_ip_bytes         0
label                 0
dtype: int64

In [10]:
df=df.dropna()

In [11]:
df.isnull().sum()

ts               0
id.orig_p        0
id.resp_p        0
proto            0
service          0
duration         0
orig_bytes       0
resp_bytes       0
conn_state       0
missed_bytes     0
history          0
orig_pkts        0
orig_ip_bytes    0
resp_pkts        0
resp_ip_bytes    0
label            0
dtype: int64

In [12]:
df.shape

(884228, 16)

Separating dependent and independent variables

In [13]:
X= df

In [14]:
Y=df['label']

In [15]:
X.shape

(884228, 16)

so we only remain with 5 classes 

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884228 entries, 98304 to 1244675
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ts             884228 non-null  float64
 1   id.orig_p      884228 non-null  float64
 2   id.resp_p      884228 non-null  float64
 3   proto          884228 non-null  object 
 4   service        884228 non-null  object 
 5   duration       884228 non-null  object 
 6   orig_bytes     884228 non-null  object 
 7   resp_bytes     884228 non-null  object 
 8   conn_state     884228 non-null  object 
 9   missed_bytes   884228 non-null  float64
 10  history        884228 non-null  object 
 11  orig_pkts      884228 non-null  float64
 12  orig_ip_bytes  884228 non-null  float64
 13  resp_pkts      884228 non-null  float64
 14  resp_ip_bytes  884228 non-null  float64
 15  label          884228 non-null  object 
dtypes: float64(8), object(8)
memory usage: 114.7+ MB


In [17]:
X['orig_bytes']=X['orig_bytes'].astype(float)

In [18]:
X['duration']=X['duration'].astype(float)

In [19]:
X['resp_bytes']=X['resp_bytes'].astype(float)

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884228 entries, 98304 to 1244675
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ts             884228 non-null  float64
 1   id.orig_p      884228 non-null  float64
 2   id.resp_p      884228 non-null  float64
 3   proto          884228 non-null  object 
 4   service        884228 non-null  object 
 5   duration       884228 non-null  float64
 6   orig_bytes     884228 non-null  float64
 7   resp_bytes     884228 non-null  float64
 8   conn_state     884228 non-null  object 
 9   missed_bytes   884228 non-null  float64
 10  history        884228 non-null  object 
 11  orig_pkts      884228 non-null  float64
 12  orig_ip_bytes  884228 non-null  float64
 13  resp_pkts      884228 non-null  float64
 14  resp_ip_bytes  884228 non-null  float64
 15  label          884228 non-null  object 
dtypes: float64(11), object(5)
memory usage: 114.7+ MB


In [22]:
X.to_csv('clean_data_new.csv')